In [3]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import gzip
import ast
import re
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

Ingesta de datos

In [8]:
steam_games= pd.read_parquet(r"C:\Users\Usuario\Desktop\HENRY\gonzalo1\df\steam_games.parquet")
user_items= pd.read_parquet(r"C:\Users\Usuario\Desktop\HENRY\gonzalo1\df\user_items.parquet")
user_reviews= pd.read_parquet(r"C:\Users\Usuario\Desktop\HENRY\pi1\duccapi1\user_reviews.parquet")


In [9]:
#Uno user_items con steam_games y elimino vacios
games_items = user_items.merge(steam_games, left_on='item_id',right_on='id',how='left')
games_items.dropna(subset=['genres'],inplace=True)
games_items.shape

(4221619, 25)

In [10]:
#Cambio de  tipo de datos
user_reviews['user_id'] = user_reviews['user_id'].str.strip()
user_items['user_id']= user_items['user_id'].str.strip()
user_reviews['item_id'] = user_reviews['item_id'].astype(float)
user_items ['item_id'] = user_items ['item_id'].astype(float) 
games_items['user_id'] = games_items ['user_id'].str.strip()

Función "countreviews"

In [17]:
# La siguiente función retorna la cantidad de usuarios que realizaron reviews y el porcentaje de reviews de estos con respecto al total de usuarios en un ranfo de fechas

def countreviews(f_inicio,f_final):
    
    #crea el rango de fechas mediante el filtro
    rango_fechas=user_reviews[(user_reviews['posted']>=f_inicio)&(user_reviews['posted']<=f_final)]
    
    #calculo de la cantidad de usuarios que hicieron reviews entre esas fechas
    count_usu= rango_fechas["user_id"].nunique()
    
    #calculo el porcentaje de reviews en el mismo rango
    porcentaje_fechas=(rango_fechas["recommend"].sum() / len(rango_fechas))* 100
    
    return{
        "Cantidad de usuarios con reseñas" : count_usu ,
        "Porcentaje de reviews entre fechas": round(float(porcentaje_fechas), 3)
    }

In [12]:
countreviews('2011-07-10', '2011-08-10')

{'Cantidad de usuarios con reseñas': 41,
 'Porcentaje de reviews entre fechas': 97.959}

Función "genre"

In [13]:
dfgenre = games_items[['genres', 'playtime_forever']]
dfgenre['genres'] = dfgenre['genres'].fillna("[]")
dfgenre['genres'] = dfgenre['genres'].apply(ast.literal_eval)

dfgenre=dfgenre.explode('genres')
dfgenre

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,genres,playtime_forever
0,Action,6.0
1,Action,0.0
2,Action,7.0
3,Action,0.0
4,Action,0.0
...,...,...
5186817,Casual,4.0
5186817,Free to Play,4.0
5186817,Indie,4.0
5186820,Action,NaN


In [14]:
dfgenre2 = dfgenre.groupby('genres',as_index=False)['playtime_forever'].sum()
dfgenre2.sort_values(by='playtime_forever',inplace=True,ascending=False)

dfgenre2['Puesto'] = range(1, len(dfgenre2) + 1)
dfgenre2.to_parquet('genre.parquet')


In [15]:
def genre(genero):
    dfgenre2= pd.read_parquet('genre')
    puesto = dfgenre2[dfgenre2['genres'] == genero]['Puesto'].iloc[0].item()
    
    return {'El género': genero, 
            'se encuentra en la posición': puesto
    }


In [16]:
genre('Education')

FileNotFoundError: [Errno 2] No such file or directory: 'genre'

Función "userforgenre"

In [ ]:
df = games_items[['genres', 'user_id', 'user_url','playtime_forever']]
df.head(3)

,genres,user_id,user_url,playtime_forever
0,['Action'],76561197970982479,http://steamcommunity.com/profiles/76561197970...,6.0
1,['Action'],76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0
2,['Action'],76561197970982479,http://steamcommunity.com/profiles/76561197970...,7.0


In [ ]:
df['genres'] = df['genres'].fillna("[]")
df['genres'] = df['genres'].apply(ast.literal_eval)
df = df.explode('genres')
df.head()

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,genres,user_id,user_url,playtime_forever
0,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,6.0
1,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0
2,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,7.0
3,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0
4,Action,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0


In [ ]:
df = df.groupby(['genres','user_id','user_url'],as_index=False)['playtime_forever'].sum()
dfufg = pd.DataFrame(df)
dfufg = dfufg.sort_values(by=['genres', 'playtime_forever'],ascending=False)
dfufg

,genres,user_id,user_url,playtime_forever
703833,Web Publishing,Xyphien,http://steamcommunity.com/id/Xyphien,142964.0
702318,Web Publishing,76561198035718256,http://steamcommunity.com/profiles/76561198035...,115083.0
704487,Web Publishing,pretentiouswutangratz,http://steamcommunity.com/id/pretentiouswutang...,105617.0
702064,Web Publishing,3197,http://steamcommunity.com/id/3197,79237.0
704381,Web Publishing,minakazu,http://steamcommunity.com/id/minakazu,75240.0
...,...,...,...,...
84503,Action,zudna,http://steamcommunity.com/id/zudna,0.0
84511,Action,zutututu,http://steamcommunity.com/id/zutututu,0.0
84525,Action,zyxif,http://steamcommunity.com/id/zyxif,0.0
84527,Action,zyzzchris,http://steamcommunity.com/id/zyzzchris,0.0


In [ ]:
dfufg.to_parquet('userforgenre.parquet')


In [ ]:
def userforgenre(genero):
 ufg= pd.read_parquet('userforgenre')
 mask = ufg[ufg['genres'] == genero]

    # Tomar los primeros 5 registros del DataFrame
 top5 = mask[['user_id', 'user_url', 'playtime_forever']].head(5).reset_index(drop=True)

 return  {'El TOP 5 de usuarios para el género' : genero, 
             'es el siguiente' : top5}

In [ ]:
userforgenre('Action')

{'El TOP 5 de usuarios para el género': 'Action',
 'es el siguiente':             user_id                                       user_url  \
 0           Sp3ctre           http://steamcommunity.com/id/Sp3ctre   
 1       shinomegami       http://steamcommunity.com/id/shinomegami   
 2      REBAS_AS_F-T      http://steamcommunity.com/id/REBAS_AS_F-T   
 3  Terminally-Chill  http://steamcommunity.com/id/Terminally-Chill   
 4   DownSyndromeKid   http://steamcommunity.com/id/DownSyndromeKid   
 
    playtime_forever  
 0         1699307.0  
 1         1580428.0  
 2         1456212.0  
 3         1065742.0  
 4         1061193.0  }